## Calculate the number of levers and levers of the secondary

In [51]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'


In [52]:
dfChild = pd.read_csv("Child.csv")
dfParent = pd.read_csv("Parent.csv")
dfOrder = pd.read_csv("Order.csv")
dfReqStockoutProductDetail = pd.read_csv("ReqStockoutProductDetail.csv") # Request Stockout Product Detail
dfReturnProduct = pd.read_csv("ReturnProduct.csv") # Return Product


In [53]:
dfOrder_drop_dup = dfOrder.drop_duplicates(subset=['order_code', 'product_code'], keep='first').reset_index(drop= True)
dfOrder_groupby_sum = dfOrder.groupby(['order_code', 'product_code'], as_index=False)['stockout_quantity'].sum()
dfOrder_groupby_sum.sort_values(by = ['stockout_quantity'])
dfOrder = pd.merge(dfOrder_drop_dup , dfOrder_groupby_sum, on=['order_code', 'product_code'], how='inner').drop(columns = ['stockout_quantity_x' , 'parent_product']).rename(columns = {"stockout_quantity_y":"stockout_quantity"})

In [54]:
def mergeMaterialOrder(dfChild, dfParent, dfOrder):
   dfChild2 = dfChild.drop_duplicates(keep='first').reset_index(drop = True)
   dfChild2 = dfChild2.sort_values(by=['parent_product']).reset_index(drop = True)
   dfChild3 = pd.merge(dfChild2 , dfParent[['parent_product','product_quantity','temporary_quantity']], left_on='child_product', right_on='parent_product', how='inner').drop(columns = ['parent_product_y']).rename(columns = {"parent_product_x":"parent_product"})
   dfChild3 = dfChild3.reset_index(drop = True)
   g = dfChild3.groupby(["parent_product"]).cumcount().add(1)
   dfChildMerge = dfChild3.set_index(["parent_product", g]).unstack(fill_value=0).sort_index(axis=1, level=1)
   dfChildMerge.columns = ["{}{}".format(a, b) for a, b in dfChildMerge.columns]

   dfChildMerge = dfChildMerge.reset_index()

   dfChildMerge = dfChildMerge.replace(np.nan, 0)
   dfTotalMate = pd.merge(dfParent, dfChildMerge, on='parent_product', how='left')
   dfTotalMate = dfTotalMate.replace(np.nan, 0)
   
   dfTotalMate = dfTotalMate.drop(dfTotalMate[(dfTotalMate['child_product1'] == 0) & (dfTotalMate['is_outsourcing'] == 1)].index).reset_index(drop = True)
   new_col1 = ['parent_product','product_code' , 'product_quantity' ,'temporary_quantity', 'is_outsourcing', 
      'product_code1', 'child_product1', 'quota1',
      'product_quantity1', 'temporary_quantity1', 'product_code2',
      'child_product2', 'quota2', 'product_quantity2',
      'temporary_quantity2', 'product_code3', 'child_product3', 'quota3',
      'product_quantity3', 'temporary_quantity3', 'product_code4',
      'child_product4', 'quota4', 'product_quantity4',
      'temporary_quantity4', 'product_code5', 'child_product5', 'quota5',
      'product_quantity5', 'temporary_quantity5']
   
   dfTotalMate= dfTotalMate.reindex(columns=new_col1)
   dfTotalMate = dfTotalMate.sort_values(by= ['parent_product']).reset_index(drop = True) 

   dfTotal = pd.merge(dfOrder , dfTotalMate, left_on='product_code', right_on='product_code', how='left')
   dfTotal = dfTotal.reset_index(drop = True)
   new_col2 = ['order_code' , 'position' , 'parent_product','product_code' ,'order_quota' ,'total_quota', 
      'production_order_quantity' , 'stockout_quantity','product_quantity',
      'temporary_quantity', 'is_outsourcing' , 'product_code1', 'child_product1', 'quota1',
      'product_quantity1', 'temporary_quantity1', 'product_code2',
      'child_product2', 'quota2', 'product_quantity2',
      'temporary_quantity2', 'product_code3', 'child_product3', 'quota3',
      'product_quantity3', 'temporary_quantity3', 'product_code4',
      'child_product4', 'quota4', 'product_quantity4',
      'temporary_quantity4', 'product_code5', 'child_product5', 'quota5',
      'product_quantity5', 'temporary_quantity5']
   
   dfTotal=dfTotal.reindex(columns=new_col2)

   # dfTotal = dfTotal[:1000] 
   dfTotalReverse = dfTotal[::-1]
   dfTotalReverse = dfTotalReverse.reset_index(drop = True)
   dfTotalReverse = dfTotalReverse.drop(dfTotalReverse[np.isnan(dfTotalReverse['parent_product']) == True].index).reset_index(drop = True)
   return dfTotalMate, dfTotalReverse

In [55]:
dfTotalMate, dfTotalReverse = mergeMaterialOrder(dfChild , dfParent, dfOrder)
dfTotalMate.head(5)
dfTotalMate[dfTotalMate['product_code'] == '5-VSTXTC-TTT-DK-EW']

,parent_product,product_code,product_quantity,temporary_quantity,is_outsourcing,product_code1,child_product1,quota1,product_quantity1,temporary_quantity1,product_code2,child_product2,quota2,product_quantity2,temporary_quantity2,product_code3,child_product3,quota3,product_quantity3,temporary_quantity3,product_code4,child_product4,quota4,product_quantity4,temporary_quantity4,product_code5,child_product5,quota5,product_quantity5,temporary_quantity5
10042,10178,5-VSTXTC-TTT-DK-EW,5686.8,-36789.26,1,9-NAMCHA-000-15-AK,7539.0,1.0,9.6,-38409.97,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0


### 1. Number Need
- Đơn hàng bán(múi + SA)
- PYC Xuất kho
- YC xuất bù LSX
- Xuất đổi trả(chỉ tính sơ cấp)

#### 1.1 Number Need + Đơn hàng bán(múi + SA)

In [56]:
dfPrimary = dfTotalReverse.loc[dfTotalReverse['is_outsourcing'] == 0][['parent_product' , 'product_code' , 'total_quota' , 'is_outsourcing']]
dfPrimaryDropDup = dfPrimary.drop_duplicates(subset=['product_code', 'parent_product'], keep='first').sort_values(by = ['parent_product']).reset_index(drop= True)
dfPrimaryDropDup

,parent_product,product_code,total_quota,is_outsourcing
0,1,1-LUOIMA-BLK-80-CM,5.80,0
1,2,1-LUOIMA-WHI-80-CM,3.03,0
2,9,1-CO7033-AMT-00-EW,5.90,0
3,10,1-CO7033-BLK-00-EV,5.90,0
4,10,1-CO7033-BLK-00-EW,5.90,0
...,...,...,...,...
1777,10313,3-INPL18-DOC-CF-BT,203.20,0
1778,10316,4-INPL21-DOC-W2-GP,57.00,0
1779,10319,3-INPL24-NAI-DU-YP,60.00,0
1780,10322,6-INPL27-465-W2-YP,80.00,0


In [57]:
dfPrimaryGroup = dfPrimary.groupby(['product_code', 'parent_product'], as_index=False)['total_quota'].sum()
dfPrimaryGroup.sort_values(by = ['total_quota'])
dfNumberNeededSum = pd.merge(dfPrimaryDropDup , dfPrimaryGroup, on=['product_code', 
                                                'parent_product'], how='inner').drop(columns = ['total_quota_x']).rename(columns = {"total_quota_y":"number_needed" ,
                                                                                        "parent_product": "child_product"}).sort_values(by = ['child_product'])
dfNumberNeededPrimary = dfNumberNeededSum[['child_product' , 'number_needed']].reset_index(drop =True)
dfNumberNeededPrimary

,child_product,number_needed
0,1,7446.34
1,2,2648.39
2,9,303.29
3,10,148.07
4,10,148.07
...,...,...
1777,10313,203.20
1778,10316,57.00
1779,10319,465.00
1780,10322,80.00


#### 1.2 Number Need + PYC Xuất kho

In [58]:
dfReqStockoutProductDetail

,product_id,quantity_req_stockout,exported_req_quantity
0,1,320.25,320.25
1,2,1121.70,1121.70
2,9,1.50,1.50
3,10,6.00,6.00
4,12,11.50,11.50
...,...,...,...
1102,10251,61.00,61.00
1103,10252,21.00,21.00
1104,10253,31.00,31.00
1105,10264,30.00,0.33


In [59]:
dfReqStockoutProductPrimary = dfReqStockoutProductDetail[dfReqStockoutProductDetail['product_id'].isin(dfPrimaryDropDup['parent_product'].to_list())].reset_index(drop = True).rename(columns= {"product_id" : "child_product"})
dfReqStockoutProductPrimary['number_need_req_stockout'] = dfReqStockoutProductPrimary['quantity_req_stockout'] - dfReqStockoutProductPrimary['exported_req_quantity']
dfReqStockoutProductPrimary

,child_product,quantity_req_stockout,exported_req_quantity,number_need_req_stockout
0,1,320.25,320.25,0.00
1,2,1121.70,1121.70,0.00
2,9,1.50,1.50,0.00
3,10,6.00,6.00,0.00
4,12,11.50,11.50,0.00
...,...,...,...,...
564,10251,61.00,61.00,0.00
565,10252,21.00,21.00,0.00
566,10253,31.00,31.00,0.00
567,10264,30.00,0.33,29.67


#### 1.3 Number Need + YC xuất bù LSX

#### 1.4 Number Need + Xuất đổi trả(chỉ tính sơ cấp)

In [60]:
dfReturnProduct

,product_id,return_request_quantity,returned_quantity
0,96,1082.9,990.2
1,3802,25000.0,22850.0
2,3825,16000.0,7315.2
3,4759,2160.0,2160.0
4,5361,1000.0,1000.0
5,7764,220.0,2.6
6,8131,30.0,15.0
7,8170,5000.0,5000.0
8,9028,20.0,20.0
9,9093,220.0,220.0


In [61]:
dfReturnProductPrimary = dfReturnProduct[dfReturnProduct['product_id'].isin(dfPrimaryDropDup['parent_product'].to_list())].reset_index(drop = True).rename(columns= {"product_id" : "child_product"})
dfReturnProductPrimary['number_need_return_product'] = dfReturnProductPrimary['return_request_quantity'] - dfReturnProductPrimary['returned_quantity']
dfReturnProductPrimary

,child_product,return_request_quantity,returned_quantity,number_need_return_product
0,96,1082.9,990.2,92.7
1,3802,25000.0,22850.0,2150.0
2,3825,16000.0,7315.2,8684.8
3,4759,2160.0,2160.0,0.0
4,5361,1000.0,1000.0,0.0
5,8170,5000.0,5000.0,0.0
6,9093,220.0,220.0,0.0
7,9168,489.0,489.0,0.0
8,9394,41.0,41.0,0.0
9,9395,234.0,234.0,0.0


### 1. Number Need Total

In [62]:
dfNumberNeed = dfNumberNeededPrimary.merge(dfReqStockoutProductPrimary, on = 'child_product', how = 'outer')
dfNumberNeed = dfNumberNeed.merge(dfReturnProductPrimary, on = 'child_product', how = 'outer')
dfNumberNeeded = dfNumberNeed[['child_product' , 'number_needed' , 'number_need_req_stockout' , 'number_need_return_product']].replace(np.NaN, 0)

dfNumberNeeded['total_number_need'] = dfNumberNeeded['number_needed'] + dfNumberNeeded['number_need_req_stockout'] + dfNumberNeeded['number_need_return_product']
 
dfNumberNeeded

,child_product,number_needed,number_need_req_stockout,number_need_return_product,total_number_need
0,1,7446.34,0.0,0.0,7446.34
1,2,2648.39,0.0,0.0,2648.39
2,9,303.29,0.0,0.0,303.29
3,10,148.07,0.0,0.0,148.07
4,10,148.07,0.0,0.0,148.07
...,...,...,...,...,...
1777,10313,203.20,0.0,0.0,203.20
1778,10316,57.00,0.0,0.0,57.00
1779,10319,465.00,0.0,0.0,465.00
1780,10322,80.00,0.0,0.0,80.00


### 2. Number needed for secondary 
- Đơn hàng bán của TC
- Phiếu yêu cầu xuất kho của TC
- YC xuất bù LSX

#### 2.1 Number needed for secondary + Đơn hàng bán của TC

In [88]:
dfSecondary = dfTotalReverse.loc[dfTotalReverse['is_outsourcing'] == 1].reset_index(drop = True)

In [89]:
# dfSecondary[dfSecondary['parent_product'] == 7764]

In [90]:
def recursivelyNumberNeed(product_id, dfTotalMate, quota, level):
    aMate = dfTotalMate.loc[np.where(dfTotalMate['parent_product'] == product_id)[0][0]] 
    if (aMate['is_outsourcing'] == 0):
        return aMate['parent_product'],quota, level
    for i in range(1, 6):
        if (aMate['quota{}'.format(i)] != 0):
            quota = quota * aMate['quota{}'.format(i)]
            level = level + 1
            product_id = aMate['child_product{}'.format(i)]
            return recursivelyNumberNeed(product_id, dfTotalMate, quota, level)

In [91]:
dict = {"parent_product": [] ,"child_product" : [] , "total_quota": [],"quota" : [] , "level" : []}

def recursivelyCalculateSecondaryNeed(dfSecondary, dfTotalMate):
    for i in range(0 , len(dfSecondary)):
        aMaterial = dfSecondary.loc[i]
        total_quota = aMaterial['total_quota']
        parent_product = aMaterial['parent_product']
        for i in range(1 , 6):
            quota = level = 2
            if(aMaterial['quota{}'.format(i)] != 0):
                quota = quota * aMaterial['quota{}'.format(i)]
                product_id = aMaterial['child_product{}'.format(i)]
                child_product , quota, level = recursivelyNumberNeed(product_id, dfTotalMate, quota , level)
                dict["parent_product"].append(parent_product)
                dict["total_quota"].append(total_quota)
                dict["child_product"].append(child_product)
                dict["quota"].append(quota)
                dict["level"].append(level)
    return dict
            
dict = recursivelyCalculateSecondaryNeed(dfSecondary, dfTotalMate)
dfSecondaryNeed = pd.DataFrame.from_dict(dict)
dfSecondaryNeed


,parent_product,child_product,total_quota,quota,level
0,6609,6607,500.0000,72.0000,2
1,3983,7824,303.0000,0.0158,2
2,8162,7684,808.7500,2.0000,2
3,124,1,6.5625,2.0000,2
4,124,122,6.5625,2.0000,2
...,...,...,...,...,...
7437,10178,7539,92.8000,2.0000,2
7438,10178,7539,128.0000,2.0000,2
7439,10178,7539,112.0000,2.0000,2
7440,10178,7539,96.0000,2.0000,2


In [99]:
dfSecondaryNeed[dfSecondaryNeed['level'] == 4]

,parent_product,child_product,total_quota,quota,level
334,8103,1,123.6,0.138000,4
407,8423,1781,128.5,0.144800,4
632,8601,1,63.6,0.134400,4
1152,8601,1,66.8,0.134400,4
1459,8686,2,63.6,0.134400,4
...,...,...,...,...,...
7408,10301,10298,50.0,2.000000,4
7415,10241,10231,100.0,0.004608,4
7431,10257,6350,280.0,2.000000,4
7433,10243,10231,56.0,0.005238,4


In [98]:
dfSecondaryNeed[dfSecondaryNeed['level'] == 5]

,parent_product,child_product,total_quota,quota,level
7242,10244,10231,41.3,0.005238,5


In [69]:
dfSecondaryNeed['number_needed_secondary'] = dfSecondaryNeed['total_quota'] * dfSecondaryNeed['quota']
dfSecondaryNeedSort = dfSecondaryNeed[['child_product' , 'number_needed_secondary']]
dfSecondaryNeedSort[dfSecondaryNeedSort['child_product'] == 10310]

,child_product,number_needed_secondary
7189,10310,7.34568
7419,10310,130.00000
7420,10310,6.02500
7425,10310,275.00000
7426,10310,1.32550
7436,10310,350.00000


In [43]:
dfOrder_drop_dup = dfSecondaryNeedSort.drop_duplicates(subset=['child_product'], keep='first').reset_index(drop= True)
dfOrder_groupby_sum = dfSecondaryNeedSort.groupby(['child_product'], as_index=False)['number_needed_secondary'].sum()
dfOrder_groupby_sum.sort_values(by = ['number_needed_secondary'])
dfSecondaryNeeded = pd.merge(dfOrder_drop_dup , dfOrder_groupby_sum, on=['child_product'], how='inner').drop(columns = ['number_needed_secondary_x']).rename(columns = {"number_needed_secondary_y":"number_needed_secondary"})
dfSecondaryNeeded = dfSecondaryNeeded.sort_values(by = ['child_product']).reset_index(drop = True)
dfSecondaryNeeded[dfSecondaryNeeded['child_product'] == 10310]
dfSecondaryNeeded

,child_product,number_needed_secondary
0,1,26182.488976
1,2,45997.330168
2,4,2117.857232
3,5,2990.107490
4,26,30.100000
...,...,...
402,10313,0.960120
403,10316,7.056600
404,10319,157.904800
405,10322,7.560000


#### 2.2 Number needed for secondary + Phiếu yêu cầu xuất kho của TC

In [44]:
dfSecondaryBase = dfSecondaryNeed[['parent_product' , 'child_product' , 'quota']]
dfSecondaryBase = dfSecondaryBase.drop_duplicates(subset=['parent_product', 'child_product'], keep='first').reset_index(drop= True)
dfSecondaryBase

,parent_product,child_product,quota
0,6609,6607,36.0000
1,3983,7824,0.0079
2,8162,7684,1.0000
3,124,1,1.0000
4,124,122,1.0000
...,...,...,...
1503,10307,10305,0.0241
1504,10312,10310,1.0000
1505,10311,10310,0.0241
1506,10329,10327,0.0241


In [20]:
dfReqStockoutProductSecondary = dfReqStockoutProductDetail[dfReqStockoutProductDetail['product_id'].isin(dfSecondary['parent_product'].to_list())].reset_index(drop = True)
dfReqStockoutChild = dfReqStockoutProductSecondary.merge(dfSecondaryBase , left_on = 'product_id' , right_on = 'parent_product' , how = 'inner').drop(columns= ['parent_product' , 'product_id']).sort_values(by = 'child_product').reset_index(drop = True)
dfReqStockoutChild['quantity_req_stockout_secondaty_needed'] = (dfReqStockoutChild['quantity_req_stockout'] - dfReqStockoutChild['exported_req_quantity']) * dfReqStockoutChild['quota']
dfReqStockoutChild

,quantity_req_stockout,exported_req_quantity,child_product,quota,quantity_req_stockout_secondaty_needed
0,5.0,5.0,1,1.0000,0.00
1,7.5,4.0,1,1.0000,3.50
2,5.0,5.0,1,1.0000,0.00
3,1.5,1.5,1,1.0000,0.00
4,0.5,0.5,1,1.0000,0.00
...,...,...,...,...,...
191,69.0,69.0,8311,0.0260,0.00
192,64.0,64.0,8311,0.0293,0.00
193,0.5,0.5,8571,1.0000,0.00
194,200.0,200.0,9719,0.0241,0.00


In [21]:
dfReqStockoutChild_drop_dup = dfReqStockoutChild.drop_duplicates(subset=['child_product'], keep='first').reset_index(drop= True)
dfReqStockoutChild__groupby_sum = dfReqStockoutChild.groupby(['child_product'], as_index=False)['quantity_req_stockout_secondaty_needed'].sum()
dfReqStockoutChild__groupby_sum.sort_values(by = ['quantity_req_stockout_secondaty_needed'])
dfReqStockoutSecondaryNeeded = pd.merge(dfReqStockoutChild_drop_dup , dfReqStockoutChild__groupby_sum, on=['child_product'], how='inner').drop(columns = ['quantity_req_stockout_secondaty_needed_x']).rename(columns = {"quantity_req_stockout_secondaty_needed_y":"quantity_req_stockout_secondaty_needed"})
dfReqStockoutSecondaryNeeded = dfReqStockoutSecondaryNeeded.sort_values(by = ['child_product']).reset_index(drop = True)
dfReqStockoutSecondaryNeeded = dfReqStockoutSecondaryNeeded[['child_product' , 'quantity_req_stockout_secondaty_needed']]
dfReqStockoutSecondaryNeeded

,child_product,quantity_req_stockout_secondaty_needed
0,1,566.4000
1,2,7.9700
2,4,-4.0000
3,5,0.0000
4,26,0.0000
...,...,...
80,8232,-1.5000
81,8311,37.0059
82,8571,0.0000
83,9719,0.0000


#### 2.3 Number needed for secondary Total

In [524]:
dfNumberSecondaryNeed = dfSecondaryNeeded.merge(dfReqStockoutSecondaryNeeded, on = 'child_product', how = 'outer')
dfNumberSecondaryNeed = dfNumberSecondaryNeed.replace(np.NaN, 0)
dfNumberSecondaryNeed

,child_product,number_needed_secondary,quantity_req_stockout_secondaty_needed
0,1,26182.488976,566.40
1,2,45997.330168,7.97
2,4,2117.857232,-4.00
3,5,2990.107490,0.00
4,26,30.100000,0.00
...,...,...,...
402,10313,0.960120,0.00
403,10316,7.056600,0.00
404,10319,157.904800,0.00
405,10322,7.560000,0.00


#### 2. Number needed total for secondary 

In [525]:
dfNumberSecondaryNeed['total_number_needed_secondary'] = dfNumberSecondaryNeed['number_needed_secondary'] + dfNumberSecondaryNeed['quantity_req_stockout_secondaty_needed']
dfNumberSecondaryNeed

,child_product,number_needed_secondary,quantity_req_stockout_secondaty_needed,total_number_needed_secondary
0,1,26182.488976,566.40,26748.888976
1,2,45997.330168,7.97,46005.300168
2,4,2117.857232,-4.00,2113.857232
3,5,2990.107490,0.00,2990.107490
4,26,30.100000,0.00,30.100000
...,...,...,...,...
402,10313,0.960120,0.00,0.960120
403,10316,7.056600,0.00,7.056600
404,10319,157.904800,0.00,157.904800
405,10322,7.560000,0.00,7.560000


### 3 Total

In [526]:
dfNeeded = dfNumberNeeded.merge(dfNumberSecondaryNeed, left_on = 'child_product', right_on= 'child_product' , how = 'outer').sort_values(by = ['child_product']).reset_index(drop = True)
dfNeeded

,child_product,number_needed,number_need_req_stockout,number_need_return_product,total_number_need,number_needed_secondary,quantity_req_stockout_secondaty_needed,total_number_needed_secondary
0,1,7446.34,0.0,0.0,7446.34,26182.488976,566.40,26748.888976
1,2,2648.39,0.0,0.0,2648.39,45997.330168,7.97,46005.300168
2,4,NaN,NaN,NaN,NaN,2117.857232,-4.00,2113.857232
3,5,NaN,NaN,NaN,NaN,2990.107490,0.00,2990.107490
4,9,303.29,0.0,0.0,303.29,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1919,10313,203.20,0.0,0.0,203.20,0.960120,0.00,0.960120
1920,10316,57.00,0.0,0.0,57.00,7.056600,0.00,7.056600
1921,10319,465.00,0.0,0.0,465.00,157.904800,0.00,157.904800
1922,10322,80.00,0.0,0.0,80.00,7.560000,0.00,7.560000


In [529]:
dfTotalNeeded = dfNeeded[['child_product' , 'total_number_need' , 'total_number_needed_secondary']].replace(np.NaN , 0)
dfTotalNeeded['total_need'] = dfTotalNeeded['total_number_need']+ dfTotalNeeded['total_number_needed_secondary']

dfTotalNeeded

,child_product,total_number_need,total_number_needed_secondary,total_need
0,1,7446.34,26748.888976,34195.228976
1,2,2648.39,46005.300168,48653.690168
2,4,0.00,2113.857232,2113.857232
3,5,0.00,2990.107490,2990.107490
4,9,303.29,0.000000,303.290000
...,...,...,...,...
1919,10313,203.20,0.960120,204.160120
1920,10316,57.00,7.056600,64.056600
1921,10319,465.00,157.904800,622.904800
1922,10322,80.00,7.560000,87.560000
